In [99]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [100]:
%sql postgresql://appdev@data/appdev

'Connected: appdev@appdev'

# Tables and columns in the f1db schema

In [ ]:
%sql SELECT * from information_schema.tables WHERE table_schema = 'f1db';

In [ ]:
%sql SELECT * from information_schema.columns WHERE table_schema = 'f1db';

# On the table *circuits* report:

## What type of indices exists for the table and why they are of that type (and not some other type)

In [41]:
%sql select * from pg_indexes where tablename = 'circuits'

3 rows affected.


schemaname,tablename,indexname,tablespace,indexdef
f1db,circuits,idx_17102_url,None,CREATE UNIQUE INDEX idx_17102_url ON circuits USING btree (url)
f1db,circuits,circuits_position_idx,None,"CREATE INDEX circuits_position_idx ON circuits USING gist (""position"")"
f1db,circuits,idx_17102_primary,None,CREATE UNIQUE INDEX idx_17102_primary ON circuits USING btree (circuitid)


As you can see there exists 3 indexes in the table *circuits*. The indexes *idx_17102_url* and *idx_17102_primary* are both of the type btree. This is probably because the indexes only need to hold integer values and the btree type is the default for indexes in PostgreSQL.
The index *circuits_position_idx* is the type gist which makes sense because the *position* value for *circuits* is coordinates. GiST trees allows us to build whichever type we want and use <key, pointer> pairs instead of integers (B-trees use integers).
So GiST trees makes it possible to index on whatever type of data we want instead of just integers for example. 

## The amount of space each index takes up

To see the amount of space each index takes up we can just write the following in a postgresql bash: 
> \di+ circuits*;

This would give us the following result: 

|Schema |         Name          | Type  | Owner  |  Table   |    Size    |
|-------|-----------------------|-------|--------|----------|------------|
|f1db   | circuits_position_idx | index | appdev | circuits | 8192 bytes |

The index *circuits_position_idx* takes up 8192 bytes. The other two indexes are not shown here for some reason. It might be because they are just default indexes. To see them I have type the following in my postgresql bash:
> \di+

This gives us the size of the last two indexes:

|Schema  |                    Name                    | Type  | Owner  |        Table         |    Size     |
|---------|--------------------------------------------|-------|--------|---------------------|------------|
|f1db    | idx_17102_primary                          | index | appdev | circuits             | 16 kB      |
|f1db    | idx_17102_url                              | index | appdev | circuits             | 16 kB      |

## We are talent scouts looking to win over some of the best new drivers there are. But we don't want them too old! Write a query that finds the winner of all the races, but only if they are younger than 38 years. The query should give return the date, driver surname, driver age, track time in milliseconds, race name and circuit name for all races.

In [109]:
%%sql SELECT races.date, drivers.surname, EXTRACT(YEAR from AGE(CURRENT_DATE, drivers.dob)) as "age", 
results.milliseconds, races.name as "race name", circuits.name as "circuit name"
FROM f1db.results 
INNER JOIN f1db.drivers ON
results.driverid = drivers.driverid
INNER JOIN f1db.races ON
results.raceid = races.raceid
INNER JOIN f1db.circuits ON
races.circuitid = circuits.circuitid
WHERE results.position = 1
AND extract(year from age(CURRENT_DATE, drivers.dob)) < 38

184 rows affected.


date,surname,age,milliseconds,race name,circuit name
2008-03-16,Hamilton,33.0,5690616,Australian Grand Prix,Albert Park Grand Prix Circuit
2008-04-06,Massa,36.0,5466970,Bahrain Grand Prix,Bahrain International Circuit
2008-05-11,Massa,36.0,5209451,Turkish Grand Prix,Istanbul Park
2008-05-25,Hamilton,33.0,7242742,Monaco Grand Prix,Circuit de Monaco
2008-06-08,Kubica,33.0,5784227,Canadian Grand Prix,Circuit Gilles Villeneuve
2008-06-22,Massa,36.0,5510245,French Grand Prix,Circuit de Nevers Magny-Cours
2008-07-06,Hamilton,33.0,5949440,British Grand Prix,Silverstone Circuit
2008-07-20,Hamilton,33.0,5480874,German Grand Prix,Hockenheimring
2008-08-03,Kovalainen,36.0,5847067,Hungarian Grand Prix,Hungaroring
2008-08-24,Massa,36.0,5732339,European Grand Prix,Valencia Street Circuit


## Describe the query using EXPLAIN ANALYZE with at least 5 lines of text. Answer at least the following:

In [108]:
%%sql EXPLAIN ANALYSE
SELECT races.date, drivers.surname, EXTRACT(YEAR from AGE(CURRENT_DATE, drivers.dob)) as "age", 
results.milliseconds, races.name as "race name", circuits.name as "circuit name"
FROM f1db.results 
INNER JOIN f1db.drivers ON
results.driverid = drivers.driverid
INNER JOIN f1db.races ON
results.raceid = races.raceid
INNER JOIN f1db.circuits ON
races.circuitid = circuits.circuitid
WHERE results.position = 1
AND extract(year from age(CURRENT_DATE, drivers.dob)) < 38

22 rows affected.


QUERY PLAN
Hash Join (cost=82.13..815.57 rows=324 width=66) (actual time=10.826..25.115 rows=184 loops=1)
Hash Cond: (races.circuitid = circuits.circuitid)
-> Hash Join (cost=78.48..803.83 rows=324 width=50) (actual time=10.121..22.904 rows=184 loops=1)
Hash Cond: (results.raceid = races.raceid)
-> Hash Join (cost=38.53..759.78 rows=324 width=27) (actual time=1.151..12.523 rows=184 loops=1)
Hash Cond: (results.driverid = drivers.driverid)
-> Seq Scan on results (cost=0.00..708.96 rows=974 width=24) (actual time=0.011..6.661 rows=974 loops=1)
"Filter: (""position"" = 1)"
Rows Removed by Filter: 22703
-> Hash (cost=35.03..35.03 rows=280 width=19) (actual time=1.121..1.121 rows=60 loops=1)


### How many calls are you making?

I'm not quite sure what "calls" are referring to in this case. But I'm guessing it means how many different operations/actions are we doing in this query. If this is the case we can simply just count the lines where an operation is used. All operations except the first will start with "->".
First we use 3 operations on joins. Then we scan on *results* for position = 1. This removes 22703 rows.
PostgreSQL then does some hashing.
Then we scan on *drivers* where their age is less than 38. This removes 781 rows.
Agian postgreSQL does some hashing.
Then a scan on *races* with hashing again.
And then last a scan on *circuits*. 
In total we make 10 "calls". 
If calls refer to how many rows each "operation" found you can simply just count all the "rows=x".

### How long does it take to perform the query?

The query uses 0.318 ms on planning time and 3.971 ms on execution time.


## Create a materialized view of your query. Using EXPLAIN ANALYZE try to query the view. Write at least 5 lines of text explaining what's going on and why the query execution time changed.

In [101]:
%%sql CREATE MATERIALIZED VIEW race_winners_cache AS
SELECT races.date, drivers.surname, EXTRACT(YEAR from AGE(CURRENT_DATE, drivers.dob)) as "age", 
results.milliseconds, races.name as "race name", circuits.name as "circuit name"
FROM f1db.results 
INNER JOIN f1db.drivers ON
results.driverid = drivers.driverid
INNER JOIN f1db.races ON
results.raceid = races.raceid
INNER JOIN f1db.circuits ON
races.circuitid = circuits.circuitid
WHERE results.position = 1
AND extract(year from age(CURRENT_DATE, drivers.dob)) < 38

184 rows affected.


[]

In [107]:
%sql EXPLAIN ANALYSE SELECT * FROM race_winners_cache

3 rows affected.


QUERY PLAN
Seq Scan on race_winners_cache (cost=0.00..4.84 rows=184 width=69) (actual time=0.012..0.818 rows=184 loops=1)
Planning time: 0.050 ms
Execution time: 1.605 ms


When creating a materialized view we cache the result of the query. That means that we don't actually have to run the query each time but instaed just save the result in cache and return the result almost instantly. This gives us a huge advantage in the amount of the it takes to retrieve the information we want but it also means that the query returns the same result every time. If the data in the database change we would get the wrong information out. We would have to manually run the "REFRESH MATERIALIZED VIEW" command on the view. 
As you can see in the result the planning and execution time has been reduced drastically which is the advantage of a materialized view. 